# AutoKeras Benchmark: HTGR Micro-Core Quadrant Power

**Input**

- `theta1`: Angle of control drum in quadrant 1 (radians) 
- `theta2`: Angle of control drum in quadrant 1 (radians) 
- `theta3`: Angle of control drum in quadrant 2 (radians)  
- `theta4`: Angle of control drum in quadrant 2 (radians)
- `theta5`: Angle of control drum in quadrant 3 (radians)
- `theta6`: Angle of control drum in quadrant 3 (radians)
- `theta7`: Angle of control drum in quadrant 4 (radians)  
- `theta8`: Angle of control drum in quadrant 4 (radians)  

**Output** 

- `fluxQ1` : Neutron flux in quadrant 1 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ2` : Neutron flux in quadrant 2 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ3` : Neutron flux in quadrant 3 ($\frac{neutrons}{cm^{2} s}$)
- `fluxQ4` : Neutron flux in quadrant 4 ($\frac{neutrons}{cm^{2} s}$)


We will be benchmarking the complete HTGR dataset of 3004 samples using H2O ML (version 3.46.0.5) in efforts to compare pyMAISE to other industry standard ML benchmarking frameworks. We will be following the same procedures we did in the original HTGR example, first extending the dataset to 3004 samples using symmetry, and then training and evaluating to compare results. Since Keras is a deep-learning framework, this benchmark will follow all the procedures we set for the FNN in the original HTGR example.

In [1]:
# Importing Packages
import time
import numpy as np
import pandas as pd

# Set display option to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Set the width of the columns
pd.set_option('display.width', None)

# See the full content of each column
pd.set_option('display.max_colwidth', None)

import xarray as xr
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, MinMaxScaler
# Plot settings
matplotlib_settings = {
    "font.size": 12,
    "legend.fontsize": 11,
    "figure.figsize": (8, 8)
}
plt.rcParams.update(**matplotlib_settings)

## Processing the data

First, we will load the raw data into a dataframe and print it out.

In [2]:
import os

cwd = os.getcwd()
new_cwd = cwd.replace("/docs/source/benchmarks", "/pyMAISE/datasets")

# Define the full path to the microreactor.csv file
csv_path = os.path.join(new_cwd, 'microreactor.csv')

# Load the CSV file into a pandas DataFrame
raw_data = pd.read_csv(csv_path)
raw_data.head()

,sample number,cpu_time,runtime,k,fluxQ1,fluxQ2,fluxQ3,fluxQ4,k_uncert,flux_runcertQ1,flux_runcertQ2,flux_runcertQ3,flux_runcertQ4,fissQ1,fissQ2,fissQ3,fissQ4,fissEQ1,fissEQ2,fissEQ3,fissEQ4,fiss_runcertQ1,fiss_runcertQ2,fiss_runcertQ3,fiss_runcertQ4,fissE_runcertQ1,fissE_runcertQ2,fissE_runcertQ3,fissE_runcertQ4,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8
0,sample_00000,4260.0,200.0,0.998328,2.580000e+19,2.590000e+19,2.670000e+19,2.560000e+19,0.00019,0.00112,0.00111,0.00111,0.00108,8.490000e+16,8.490000e+16,8.480000e+16,8.490000e+16,2751290,2751060,2749270,2750450,0.00060,0.00060,0.00063,0.00062,0.00060,0.00060,0.00063,0.00062,5.919526,2.369503,2.923656,4.488987,3.683212,4.008905,4.970368,2.987966
1,sample_00001,2570.0,130.0,0.988522,2.550000e+19,2.530000e+19,2.510000e+19,2.510000e+19,0.00025,0.00142,0.00148,0.00154,0.00150,8.490000e+16,8.490000e+16,8.490000e+16,8.490000e+16,2750610,2750210,2750150,2750110,0.00076,0.00077,0.00084,0.00074,0.00076,0.00077,0.00084,0.00074,2.162380,0.273624,0.927741,4.595586,2.598824,0.170167,2.124048,4.980209
2,sample_00002,2590.0,130.0,1.004610,2.570000e+19,2.580000e+19,2.520000e+19,2.520000e+19,0.00025,0.00167,0.00163,0.00161,0.00165,8.480000e+16,8.480000e+16,8.490000e+16,8.490000e+16,2748870,2749690,2752250,2751840,0.00076,0.00077,0.00086,0.00080,0.00076,0.00077,0.00086,0.00080,0.450100,0.006301,2.512217,3.313864,1.913458,3.582252,0.280764,4.888595
3,sample_00003,2580.0,129.0,0.991892,2.570000e+19,2.580000e+19,2.520000e+19,2.560000e+19,0.00025,0.00197,0.00193,0.00195,0.00200,8.480000e+16,8.490000e+16,8.480000e+16,8.470000e+16,2748920,2750720,2749330,2746220,0.00082,0.00076,0.00080,0.00078,0.00082,0.00076,0.00080,0.00078,0.461105,4.825628,3.771356,2.599278,2.056019,0.007332,1.106786,5.504671
4,sample_00004,2570.0,129.0,0.985047,2.540000e+19,2.620000e+19,2.580000e+19,2.520000e+19,0.00025,0.00167,0.00167,0.00172,0.00169,8.480000e+16,8.490000e+16,8.480000e+16,8.490000e+16,2748910,2753130,2747870,2752420,0.00080,0.00081,0.00082,0.00083,0.00080,0.00081,0.00082,0.00083,5.248202,3.549416,3.333632,3.907310,2.095312,5.585145,3.774253,2.480120


We are then going to create input and output dataframes by defining our input and output variables.

In [3]:
# Create the input DataFrame with theta values
input_columns = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8']
inputs = raw_data[input_columns]

# Create the output DataFrame with flux values
output_columns = ['fluxQ1', 'fluxQ2', 'fluxQ3', 'fluxQ4']
outputs = raw_data[output_columns]

Below, we print out the results for input and output then also create a combined dataset with both.

In [4]:
inputs.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8
0,5.919526,2.369503,2.923656,4.488987,3.683212,4.008905,4.970368,2.987966
1,2.162380,0.273624,0.927741,4.595586,2.598824,0.170167,2.124048,4.980209
2,0.450100,0.006301,2.512217,3.313864,1.913458,3.582252,0.280764,4.888595
3,0.461105,4.825628,3.771356,2.599278,2.056019,0.007332,1.106786,5.504671
4,5.248202,3.549416,3.333632,3.907310,2.095312,5.585145,3.774253,2.480120


In [5]:
outputs.head()

,fluxQ1,fluxQ2,fluxQ3,fluxQ4
0,2.580000e+19,2.590000e+19,2.670000e+19,2.560000e+19
1,2.550000e+19,2.530000e+19,2.510000e+19,2.510000e+19
2,2.570000e+19,2.580000e+19,2.520000e+19,2.520000e+19
3,2.570000e+19,2.580000e+19,2.520000e+19,2.560000e+19
4,2.540000e+19,2.620000e+19,2.580000e+19,2.520000e+19


In [6]:
combined_df = pd.concat([inputs, outputs], axis=1)
print(combined_df.head())

     theta1    theta2    theta3    theta4    theta5    theta6    theta7  \
0  5.919526  2.369503  2.923656  4.488987  3.683212  4.008905  4.970368   
1  2.162380  0.273624  0.927741  4.595586  2.598824  0.170167  2.124048   
2  0.450100  0.006301  2.512217  3.313864  1.913458  3.582252  0.280764   
3  0.461105  4.825628  3.771356  2.599278  2.056019  0.007332  1.106786   
4  5.248202  3.549416  3.333632  3.907310  2.095312  5.585145  3.774253   

     theta8        fluxQ1        fluxQ2        fluxQ3        fluxQ4  
0  2.987966  2.580000e+19  2.590000e+19  2.670000e+19  2.560000e+19  
1  4.980209  2.550000e+19  2.530000e+19  2.510000e+19  2.510000e+19  
2  4.888595  2.570000e+19  2.580000e+19  2.520000e+19  2.520000e+19  
3  5.504671  2.570000e+19  2.580000e+19  2.520000e+19  2.560000e+19  
4  2.480120  2.540000e+19  2.620000e+19  2.580000e+19  2.520000e+19  


Now it is time to extend the dataset to 3004 samples. This is done in the same way as in the original HTGR, replicating the same steps below.

In [7]:
# Credit to mult_sym and g21 from https://github.com/deanrp2/MicroControl/blob/main/pmdata/utils.py#L51
theta_cols = [f"theta{i + 1}" for i in range(8)]
flux_cols = [f"fluxQ{i + 1}" for i in range(4)]

def mult_samples(data):
    # Create empty arrays
    ht = xr.DataArray(
        np.zeros(data.shape), 
        coords={
            "index": [f"{idx}_h" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )
    vt = xr.DataArray(
        np.zeros(data.shape), 
        coords={
            "index": [f"{idx}_v" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )
    rt = xr.DataArray(
        np.zeros(data.shape),     
        coords={
            "index": [f"{idx}_r" for idx in data.coords["index"].values],
            "variable": data.coords["variable"],
        },
    )

    # Swap drum positions
    hkey = [f"theta{i}" for i in np.array([3, 2, 1, 0, 7, 6, 5, 4], dtype=int) + 1]
    vkey = [f"theta{i}" for i in np.array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int) + 1]
    rkey = [f"theta{i}" for i in np.array([4, 5, 6, 7, 0, 1, 2, 3], dtype=int) + 1]

    ht.loc[:, hkey] = data.loc[:, theta_cols].values
    vt.loc[:, vkey] = data.loc[:, theta_cols].values
    rt.loc[:, rkey] = data.loc[:, theta_cols].values

    # Adjust angles
    ht.loc[:, hkey] = (3 * np.pi - ht.loc[:, hkey].loc[:, hkey]) % (2 * np.pi)
    vt.loc[:, vkey] = (2 * np.pi - vt.loc[:, hkey].loc[:, vkey]) % (2 * np.pi)
    rt.loc[:, rkey] = (np.pi + rt.loc[:, hkey].loc[:, rkey]) % (2 * np.pi)

    # Fill quadrant tallies
    hkey = [2, 1, 4, 3]
    vkey = [4, 3, 2, 1]
    rkey = [3, 4, 1, 2]

    ht.loc[:, [f"fluxQ{i}" for i in hkey]] = data.loc[:, flux_cols].values
    vt.loc[:, [f"fluxQ{i}" for i in vkey]] = data.loc[:, flux_cols].values
    rt.loc[:, [f"fluxQ{i}" for i in rkey]] = data.loc[:, flux_cols].values

    sym_data = xr.concat([data, ht, vt, rt], dim="index").sortby("index")
    
    # Normalize fluxes
    sym_data.loc[:, flux_cols].values = Normalizer().transform(sym_data.loc[:, flux_cols].values)
    
    # Convert global coordinate system to local
    loc_offsets = np.array(
        [3.6820187359906447, 4.067668586955522, 2.2155167202240653 - np.pi, 2.6011665711889425 - np.pi, 
         0.5404260824008517, 0.9260759333657285, 5.3571093738138575 - np.pi, 5.742759224778734 - np.pi]
    )

    # Apply correct 0 point
    sym_data.loc[:, theta_cols] = sym_data.loc[:, theta_cols] - loc_offsets + 2 * np.pi

    # Reverse necessary angles
    sym_data.loc[:, [f"theta{i}" for i in [3,4,5,6]]] *= -1

    # Scale all to [0, 2 * np.pi]
    sym_data.loc[:, theta_cols] = sym_data.loc[:, theta_cols] % (2 * np.pi)
        
    return sym_data

In [8]:
train_data, test_data = train_test_split(combined_df, test_size=0.3)

# Convert to xarray DataArray and specify the index as a coordinate
train_data_xr = xr.DataArray(
    train_data.values,
    coords={"index": train_data.index, "variable": train_data.columns},
    dims=["index", "variable"]
)
test_data_xr = xr.DataArray(
    test_data.values,
    coords={"index": test_data.index, "variable": test_data.columns},
    dims=["index", "variable"]
)

In [9]:
sym_train_data = mult_samples(train_data_xr)
sym_test_data = mult_samples(test_data_xr)
print(f"Multiplied training shape: {sym_train_data.shape}, Multiplied testing shape: {sym_test_data.shape}")

Multiplied training shape: (2100, 12), Multiplied testing shape: (904, 12)


As seen above, we end up with data the same size as the original HTGR. Below, we are going to Min-Max the X_data and normalize the y_data.

In [10]:
# Min-Max scaling data 
def scale_data(train_data, test_data, scaler):
    train_data.values = scaler.fit_transform(
        train_data.values.reshape(-1, train_data.shape[-1])
    ).reshape(train_data.shape)
    test_data.values = scaler.transform(
        test_data.values.reshape(-1, test_data.shape[-1])
    ).reshape(test_data.shape)
    
    # Return data
    return train_data, test_data, scaler

xtrain_arr, xtest_arr , _ = scale_data(sym_train_data.loc[:, theta_cols], sym_test_data.loc[:, theta_cols], MinMaxScaler())
ytrain_arr, ytest_arr, _ = scale_data(sym_train_data.loc[:, flux_cols], sym_test_data.loc[:, flux_cols], Normalizer(norm="l1"))

In [11]:
xtrain = xtrain_arr.to_pandas()
xtest = xtest_arr.to_pandas()
ytrain = ytrain_arr.to_pandas()
ytest = ytest_arr.to_pandas()

## Benchmark with AutoKeras

After preprocessing, we are going to now train an AutoKeras model on the data. First, we will import the necessary libraries. We will be using the CPU only for these tasks since we did the same for HTGR.

In [12]:
#Import things required from Keras
import autokeras as ak
import tensorflow as tf
import keras_tuner
import tensorflow.keras.backend as K

Using TensorFlow backend


2024-10-24 15:23:38.896153: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 15:23:38.927502: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 15:23:38.928279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 15:23:39.453443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Below we define the R2 metric so that we can train our AutoKeras model to maximize validation R2 score. We will use a bayesian tuner as in the pyMAISE example and also use MSE as our loss. We are going to try a max of 50 models.

In [13]:
# Custom R2 metric
def r2_score(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

In [14]:
regressor = ak.StructuredDataRegressor(
    max_trials=50, 
    overwrite=True,
    loss='mean_squared_error',
    directory='HTGR_Keras_model',
    metrics=[r2_score, "mean_absolute_error", "mean_squared_error", "mean_absolute_percentage_error"],
    objective=keras_tuner.Objective('val_r2_score', direction='max'),
    tuner='bayesian',
)

We are going to train our training dataset, setting epochs to 50 as in the pyMAISE example.

In [15]:
regressor.fit(xtrain, ytrain, epochs=50)

Trial 50 Complete [00h 00m 02s]
val_r2_score: 0.19001974165439606

Best val_r2_score So Far: 0.8381330966949463
Total elapsed time: 00h 03m 11s
Epoch 1/50
66/66 [==============================] - 0s 877us/step - loss: 0.0118 - r2_score: -588.4028 - mean_absolute_error: 0.0678 - mean_squared_error: 0.0118 - mean_absolute_percentage_error: 27.1121
Epoch 2/50
66/66 [==============================] - 0s 702us/step - loss: 3.0753e-04 - r2_score: -17.7216 - mean_absolute_error: 0.0135 - mean_squared_error: 3.0753e-04 - mean_absolute_percentage_error: 5.3787
Epoch 3/50
66/66 [==============================] - 0s 643us/step - loss: 4.2264e-05 - r2_score: -1.4148 - mean_absolute_error: 0.0051 - mean_squared_error: 4.2264e-05 - mean_absolute_percentage_error: 2.0216
Epoch 4/50
66/66 [==============================] - 0s 674us/step - loss: 1.8311e-05 - r2_score: -0.0337 - mean_absolute_error: 0.0033 - mean_squared_error: 1.8311e-05 - mean_absolute_percentage_error: 1.3298
Epoch 5/50
66/66 [======

INFO:tensorflow:Assets written to: HTGR_Keras_model/structured_data_regressor/best_model/assets


Now that the best model was chosen, we are going to load that model in and train for another 250 epochs, totaling 300 for the best model (as done in the original HTGR benchmark).

In [18]:
#Train another 250 epochs ontop of the 50 beforehand using the best model
best_model = tf.keras.models.load_model('./HTGR_Keras_model/structured_data_regressor/best_model', custom_objects={'r2_score': r2_score})
best_model.fit(xtrain, ytrain, epochs=250)

Epoch 1/250
66/66 [==============================] - 0s 823us/step - loss: 1.1730e-05 - r2_score: 0.3397 - mean_absolute_error: 0.0026 - mean_squared_error: 1.1730e-05 - mean_absolute_percentage_error: 1.0311
Epoch 2/250
66/66 [==============================] - 0s 837us/step - loss: 2.0769e-05 - r2_score: -0.1766 - mean_absolute_error: 0.0035 - mean_squared_error: 2.0769e-05 - mean_absolute_percentage_error: 1.3966
Epoch 3/250
66/66 [==============================] - 0s 850us/step - loss: 2.1443e-05 - r2_score: -0.2214 - mean_absolute_error: 0.0037 - mean_squared_error: 2.1443e-05 - mean_absolute_percentage_error: 1.4618
Epoch 4/250
66/66 [==============================] - 0s 844us/step - loss: 2.3893e-05 - r2_score: -0.3473 - mean_absolute_error: 0.0039 - mean_squared_error: 2.3893e-05 - mean_absolute_percentage_error: 1.5590
Epoch 5/250
66/66 [==============================] - 0s 841us/step - loss: 2.7886e-05 - r2_score: -0.5823 - mean_absolute_error: 0.0042 - mean_squared_error: 2.7

Now that the best model is fully trained, we can predict on our testing dataset and generate the results below. We can see that we obtain very similar results as the FNN in the original example with r2 for both around 0.97. The same can be said about the other metrics such as MSE.

In [19]:
# Evaluate the model on the test data
results = best_model.evaluate(xtest, ytest)

29/29 [==============================] - 0s 682us/step - loss: 4.4791e-07 - r2_score: 0.9724 - mean_absolute_error: 5.3476e-04 - mean_squared_error: 4.4791e-07 - mean_absolute_percentage_error: 0.2140
